In [1]:
!pip install mlflow ipywidgets transformers huggingface_hub --quiet
from google.colab import output
output.enable_custom_widget_manager()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import time
import mlflow
import calendar
import re
from transformers import pipeline
from huggingface_hub import hf_hub_download
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)

# Set MLflow remote URI
mlflow.set_tracking_uri("http://3.145.180.106:5000")
mlflow.set_experiment("Cincinnati Crime Chatbot")

def load_crime_data():
    csv_path = hf_hub_download(
        repo_id="mlsystemsg1/cincinnati-crime-data",
        repo_type="dataset",
        filename="calls_for_service_latest.csv"
    )
    df = pd.read_csv(csv_path, low_memory=False)
    df.columns = [col.lower() for col in df.columns]
    df.dropna(subset=['sna_neighborhood'], inplace=True)
    df['create_time_incident'] = pd.to_datetime(df['create_time_incident'], errors='coerce')
    return df

def get_relevant_rows(question, df):
    q = question.lower()
    filtered = df.copy()

    # Neighborhood match
    for hood in df['sna_neighborhood'].dropna().unique():
        if hood.lower() in q:
            filtered = filtered[filtered['sna_neighborhood'].str.lower() == hood.lower()]
            break

    # Offense match (any of the 3 offense columns)
    offense_cols = ['disposition_text', 'incident_type_desc', 'incident_type_id']
    for col in offense_cols:
        if col in df.columns:
            for offense in df[col].dropna().astype(str).unique():
                if offense.lower() in q:
                    filtered = filtered[filtered[col].astype(str).str.lower() == offense.lower()]
                    break

    # Exact date (YYYY-MM-DD)
    date_match = re.search(r"(20\\d{2})[-/](\\d{1,2})[-/](\\d{1,2})", q)
    if date_match:
        y, m, d = map(int, date_match.groups())
        filtered = filtered[filtered['create_time_incident'].dt.date == pd.Timestamp(y, m, d).date()]
        return filtered

    # Month match
    months = {month.lower(): i for i, month in enumerate(calendar.month_name) if month}
    for name, num in months.items():
        if name in q:
            filtered = filtered[filtered['create_time_incident'].dt.month == num]
            break

    # Year match
    year_match = re.search(r"(20\\d{2})", q)
    if year_match:
        filtered = filtered[filtered['create_time_incident'].dt.year == int(year_match.group(1))]

    # Sort if asking for most recent/latest offense
    if any(x in q for x in ["latest", "most recent", "last offense", "recent"]):
        filtered = filtered.sort_values(by='create_time_incident', ascending=False)

    # Drop rows with missing date values (NaT)
    filtered = filtered.dropna(subset=['create_time_incident'])

    return filtered

def generate_summary(question, df):
    if df.empty:
        return "No matching records found."

    if any(word in question.lower() for word in ["how many", "count", "number of"]):
        return f"There were {len(df)} incidents matching your query."

    examples = []
    for _, row in df.head(10).iterrows():
        date = row.get('create_time_incident', 'N/A')
        try:
            date_obj = pd.to_datetime(date)
            date_str = date_obj.date()
        except:
            date_str = date

        offense = row.get('incident_type_desc', 'N/A')
        hood = row.get('sna_neighborhood', 'N/A')
        incident = row.get('event_number', 'N/A')
        priority = row.get('priority', 'N/A')

        examples.append(f"On {date_str}, a {offense} (Priority {priority}) occurred in {hood} (Incident #{incident}).")

    return "\n".join(examples)

def answer_with_llm(question, data_rows, model, model_name="google/flan-t5-small", prompt_version="v5"):
    if data_rows.empty:
        return "Sorry, I couldn't find any data matching that question."
    context = generate_summary(question, data_rows)
    prompt = f"""
You are a helpful assistant analyzing Cincinnati crime data.

Here are some relevant data points:
{context}

Now answer this question based on the above:
{question}
    """.strip()
    start_time = time.time()
    with mlflow.start_run():
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("question", question)
        mlflow.log_param("num_rows_context", len(data_rows))
        mlflow.log_param("prompt_version", prompt_version)
        mlflow.log_text(prompt, "prompt.txt")
        result = model(prompt, max_new_tokens=150)[0]['generated_text']
        answer = result.strip()
        latency = time.time() - start_time
        mlflow.log_text(answer, "answer.txt")
        mlflow.log_metric("response_length", len(answer))
        mlflow.log_metric("latency_sec", latency)
        mlflow.log_metric("match_count", len(data_rows))
        return answer

def run_single_chatbot_turn(df, model, question):
    if question.lower() in ['exit', 'quit']:
        return "Goodbye!"
    rows = get_relevant_rows(question, df)
    return answer_with_llm(question, rows, model)

def ask_question_with_widget(df, model):
    input_box = widgets.Text(
        placeholder='Ask a question about crime in Cincinnati...',
        description='Question:',
        layout=widgets.Layout(width='100%')
    )
    output = widgets.Output()
    def on_submit(change):
        output.clear_output()
        question = change['new']
        with output:
            print("Bot:", run_single_chatbot_turn(df, model, question))
    input_box.observe(on_submit, names='value')
    display(input_box, output)

# Launch everything
print("Loading dataset and model...")
df = load_crime_data()
print("Columns in dataset:", df.columns.tolist())
llm_model = pipeline("text2text-generation", model="google/flan-t5-small")
print("\n✅ Chatbot is ready! Ask your question below:")
ask_question_with_widget(df, llm_model) # Ensure create_time_incident is properly parsed as datetime
df['create_time_incident'] = pd.to_datetime(df['create_time_incident'], errors='coerce')

# Now you can safely get max/min
print("Most recent incident date in dataset:")
print(df['create_time_incident'].max())

print("\nOldest incident date:")
print(df['create_time_incident'].min())

print("\nNumber of missing date entries:")
print(df['create_time_incident'].isna().sum())


Loading dataset and model...


calls_for_service_latest.csv:   0%|          | 0.00/529M [00:00<?, ?B/s]

Columns in dataset: ['create_time_incident', 'incident_type_desc', 'incident_type_id', 'disposition_text', 'event_number', 'sna_neighborhood', 'priority']


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Device set to use cpu



✅ Chatbot is ready! Ask your question below:


Text(value='', description='Question:', layout=Layout(width='100%'), placeholder='Ask a question about crime i…

Output()

Most recent incident date in dataset:
2021-09-20 00:44:18

Oldest incident date:
2014-09-30 23:52:08

Number of missing date entries:
130


In [3]:
# Ensure create_time_incident is properly parsed as datetime
df['create_time_incident'] = pd.to_datetime(df['create_time_incident'], errors='coerce')

# Now you can safely get max/min
print("Most recent incident date in dataset:")
print(df['create_time_incident'].max())

print("\nOldest incident date:")
print(df['create_time_incident'].min())

print("\nNumber of missing date entries:")
print(df['create_time_incident'].isna().sum())


Most recent incident date in dataset:
2021-09-20 00:44:18

Oldest incident date:
2014-09-30 23:52:08

Number of missing date entries:
130
